In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import poisson, bernoulli, multivariate_normal, entropy

from utils import color_cycle

# Lecture 1: Probability, entropy and inference

* **1.2 Entropy and variational approximation**
    * Entropy of distributions
    * Maximum entropy
    * Variational approximation

MacKay, **Chapter 2** (sections 2.4, 2.5, 2.6, 2.7).

_Recommended reading_:
* MacKay, **Chapter 1**
* [Wikipedia entry](https://en.wikipedia.org/wiki/Exponential_family) for exponential families
* [These notes](notes/exponential_families.pdf) from John Duchi

# What is information

**Information** is a measure of the _degree of surprise_ that a certain value gives us **given that we know the distribution**.

Unlikely events are informative, likely events less → information decreases with the probability of the event.

Given $p(x)=\delta(x)$ (no uncertainty), observing $x$ gives us no additional information. 

Let us denote $h(x)$ the information of $x$. Then if $x,y$ are independent, we want information to be additive $h(x,y)=h(x)+h(y)$. Since $p(x,y)=p(x)p(y)$ we see that
$$
h(x)=-\log_2 p(x)
$$
is a good candidate to quantify the information in $x$.

In [ ]:
# visualize the candidate information/suprise function
ps = np.linspace(1e-10, 1, 500)
plt.plot(ps, -np.log2(ps));
plt.xlabel('p')
plt.ylabel('$log_2(1/p)$');

The **expected information** is
$$
H[x]:= -\sum_x p(x)\log_2 p(x)
$$
is the **entropy** of the distribution $p$.

### Some examples

When $p$ is sharply peaked ($p(x_1)=1, p(x_2)=\ldots=p(x_M)=0$) then the
entropy is
$$
H[x]=-1\log 1 - (M-1) 0 \log 0 = 0
$$
When $p$ is flat ($p(x_i)=1/M$) the entropy is maximal
$$
H[x]=-M \frac{1}{M}\log \frac{1}{M}=\log M
$$

Let's look at how the entropy of a Poisson distribution changes with its rate

In [ ]:
# generate probabilities from Poisson distributions with different rates
ks = np.arange(100)
bins = np.arange(-0.5,100,1)

num_samples = 5000
mus = [5, 20, 40, 60]
Hs = np.zeros(len(mus))
Hs_scipy = np.zeros(len(mus))

plt.figure(figsize=(12,4))
plt.subplot(121)
for imu, mu in enumerate(mus):
    # compute probs
    ps = poisson.pmf(ks, mu=mu)
    
    # compute entropy
    H = -(ps * np.log(ps)).sum() # do it yourself
    H_scipy = entropy(ps) # be lazy and let scipy do it for you
    Hs[imu] = H
    Hs_scipy[imu] = H_scipy
    
    # extract samples and plot distributions
    samples = poisson.rvs(mu, size=num_samples);
    plt.plot(ps, '-', c=color_cycle[imu], ms=3);
    plt.hist(samples, bins=bins, density=True, color=color_cycle[imu], alpha=0.5, label="$\mu$={:d}, H = {:.3g}".format(mu, H));

plt.xlabel('k')
plt.ylabel('p(k)')
plt.legend();

plt.subplot(122)
plt.plot(mus, Hs, '.-')
plt.plot(mus, Hs_scipy, '.:') # check calculation against scipy
plt.xlabel("$\mu$")
plt.ylabel("H");

## Entropy and information transmission

Suppose we want to send symbols $x$ over a **channel**.

Symbols are assumed to be randomly generated by a distribution $p(x)$: common symbols (large $p(x)$) are sent very often, other symbols (small $p(x)$) are rarely sent.

We want to encode each symbol as a binary string: we want to use short strings for common symbols and longer strings for rare symbols.

The question is: how to do this optimally? 

#### <center>Noiseless coding theorem </center>
Entropy is a lower bound on the average number of bits per symbol (Shannon 1948).

### Example 1

$x$ can have 8 values with equal probability, then $H(x)=-8\times
\frac{1}{8}\log \frac{1}{8}=3$. We say that the distribution carries 3 bits of information. 

By Shannon's theorem, we need at least 3 bits per symbol. In fact, we can use the normal binary representation of $x$ to use exactly 3 bits per symbol.:

$x$ |  $-\log p(x)$ | code |
| -------- | ------- | ------- |
0	|	3	| 000 |
1	|	3	| 001 |
2	|	3	| 010 |
3	|	3	| 011 |
4	|	3	| 100 |
5	|	3	| 101 |
6	|	3	| 110 |
7	|	3	| 111 |

### Example 2

$x$ can have 8 values with probabilities 
$(\frac{1}{2},\frac{1}{4},\frac{1}{8},\frac{1}{16},\frac{1}{64},\frac{1}{64},\frac{1}{64},\frac{1}{64})$

Then 
$$
H(x)=-\frac{1}{2}\log\frac{1}{2}-\frac{1}{4}\log\frac{1}{4}-\frac{1}{8}\log\frac{1}{8}-\frac{1}{16}\log\frac{1}{16}-\frac{4}{64}\log\frac{1}{64}=2
\mathrm{bits}
$$

By Shannon's theorem we need on average at least two bits per symbol. The naive code as before would use 3 bits.

#### Can we construct a code that saturates the Shannon bound and only uses on average 2 bits per symbol? 

The idea is to encode each symbol $x$ with a string with length equal to its information $-\log p(x)$. 
Thus, for instance:

$x$ |  $-\log p(x)$ | code |
| -------- | ------- | ------- |
0	|	1	|  0 |
1	|	2	| 10 |
2	|	3	| 110 |
3	|	4	| 1110 |
4	|	6	| 111100 |
5	|	6	| 111101 |
6	|	6	| 111110 |
7	|	6	| 111111 |

Then by construction:
$$
\mathrm{Average\;code
length}=\sum_x p(x) \text{length}(x) = -\sum_x p(x) \log_2 p(x) = H
$$

## Maximum Entropy and Exponential Families

The **maximum entropy** approach allows us to construct distribution $p$ to describe data that make the least amount of assumptions on the data itself, apart from including some _statistics_ from the data.

Denote the data as $D=(x_1,\ldots,x_N)$. From the data, we can compute certain <em>statistics</em>
$$
S_a=\frac{1}{N}\sum_{i=1}^N \phi_a(x_i)
$$
that are averages over the data. 

Useful statistics might be the mean and variance of the data
$$
\bar{x}=\frac{1}{N}\sum_{i=1}^N x_i \qquad \sigma^2_x=\frac{1}{N}\sum_{i=1}^N \left(x_i -\bar{x}\right)^2=\frac{1}{N}\sum_i x_i^2 -\bar{x}^2
$$
$$
S_1=\bar{x}\qquad \phi_1(x)=x\qquad S_2=\sigma^2_x+\bar{x}^2\qquad \phi_2(x)=x^2
$$

We want to find a distribution $p$ that reproduces these statistics correctly. That is to say, $p$ should be such that
$$
\mathbb{E} \phi_a =\sum_x p(x) \phi_a(x) = S_a\qquad a=1,2,\ldots
$$
but in general this will not determine $p$ uniquely. 

The maximum entropy solution is the **least informative** distribution consistent with the constraints. Thus, we wish to find 
$$
\min_{p(x)\;\text{s.t.}\;\mathbb{E} \phi_a = S_a} \int dx p(x) \log p(x)
$$
where $a=0$ is the normalization constraint $\sum_x p(x) =1$.

This is a constrained optimization problem, that we can solve with **Lagrange multipliers**.

### <em>Intermezzo: Lagrange multipliers</em>

Minimize $f(x)$ with respect to $x$ under constraints $g_a(x) = 0$, $a=1,...,M$:
$$
\min_{x\;\text{s.t.}\;g_1(x)=0,\ldots,g_M(x)=0} f(x)
$$

Define the **Lagrangian**,
$$
L\left(x,\lambda\right) = f(x) +\sum_{a=1}^M \lambda_a g_a(x)
$$
$\lambda_a$ is called a Lagrange multiplier.

Find stationarity condition of $L\left(x,\lambda\right)$ w.r.t. $x$ and $\lambda$.

More generally:
$$
\min_{x\;\text{s.t.}\;g_1(x)=0,\ldots,g_M(x)=0} f(x)\qquad\rightarrow \qquad \min_x \max_{\lambda_1,\ldots,\lambda_M} f(x) +\sum_{a=1}^M \lambda_a g_a(x)
$$

### A concrete example:

Minimize

$f(x_1,x_2) =  x_1^2 + x_2^2$

subject to

$g(x_1,x_2)  =  x_1 + x_2 -1=0$

In [ ]:
# compute function values in the plane
f = lambda x, y : x**2 + y**2
l, r, dx = -3, 3, 0.1
x = np.arange(l,3,0.1)
X, Y = np.meshgrid(x, x)
F = f(X,Y)

# parametrize line
x_line = np.arange(-2, 3, 0.01)
y_line = -x_line + 1

# plot countours and constraints
plt.imshow(F, alpha=0.5, extent=[-3,3,-3,3], cmap="coolwarm");
plt.contour(X, Y, F)
plt.hlines(y=0, xmin=-3, xmax=3, ls=":", color="gray")
plt.vlines(x=0, ymin=-3, ymax=3, ls=":", color="gray")
plt.scatter(x_line, y_line, c="black", edgecolors='none', s=3)
plt.xlabel('x')
plt.ylabel('y');

Lagrangian:

$L(x_1,x_2,\lambda)  =   x_1^2 +x_2^2 + \lambda ( x_1 + x_2 -1)$

1. Maximize $L$ w.r.t. $x_{1,2}$ gives
$x_1(\lambda)  = x_2(\lambda)=-\frac{1}{2}\lambda$.

2. Plug into constraint: $
x_1(\lambda) + x_2(\lambda) - 1$ yields $\lambda^* =-1$.


The solution is $x_1^*=x_1(\lambda^*)=\frac{1}{2}$ and $x_1^*=x_1(\lambda^*)=\frac{1}{2}$

In [ ]:
# evaluate function on the line x + y = 1
f_line = f(x_line, y_line)

# plot solution in 2d
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.imshow(F, alpha=0.5, extent=[-3,3,-3,3], cmap="coolwarm");
plt.contour(X, Y, F)
plt.hlines(y=0, xmin=-3, xmax=3, ls=":", color="gray")
plt.vlines(x=0, ymin=-3, ymax=3, ls=":", color="gray")
plt.scatter(x_line, y_line, c=f_line / f_line.max(), edgecolors='none', cmap="coolwarm", s=3)
plt.plot(0.5, f(0.5, -0.5 + 1.), 'o', c="black")
plt.xlabel('x')
plt.ylabel('y')

# plot value over the line
plt.subplot(122)
plt.plot(0.5, f(0.5, -0.5 + 1.), 'o', c="black")
plt.scatter(x_line, f_line, c=f_line / f_line.max(), edgecolors='none', cmap="coolwarm", s=3)
plt.xlabel('x')
plt.ylabel('f(x,y)')

plt.tight_layout();

### Solving for Maximum Entropy distribution through Lagrange Multipliers

The Lagrangian for the maximum entropy problem takes the form
$$
L(p,\lambda) = \sum_x p(x)\log p(x) + \sum_{a=0}^M \lambda_a \left(\sum_x p(x) \phi_a(x)-S_a\right)
$$
Stationarity $L$ w.r.t. $p(x)$ yields
$$
\partial_{p\left(x\right)}L\left(p,\lambda \right)=0\implies\log p\left(x\right)+1+\lambda_{0}+\sum_{a=1}^{M}\lambda_{a}\phi_{a}\left(x\right)=0
$$
from which
$$p=e^{-1-\lambda_{0}}\exp\left(-\sum_{a=1}^{M}\lambda_{a}\phi_{a}\left(x\right)\right)$$

The factor in front is fixed by the normalization condition $\sum_x p(x)=1$. Thus calling $A=\lambda_0+1$ we have:
$$
p(x)=\exp\left(-\sum_{a=1}^M \lambda_a \phi_a(x)-A \right)\qquad A=\log \sum_{x} \exp\left(-\sum_{a=1}^M \lambda_a \phi_a(x)\right)
$$
This is called the **exponential family** model and $A$ is the **log partition sum**.

The other $\lambda_a$ are determined implicitly by the equations
$$\sum_x p(x) \phi_a(x)=S_a$$
that impose that _expectation_ of the functions $\phi_a$ over the model distribution $p$ are the same as _the empirical average_ $S_a$.

### Examples of exponential family distributions: Bernoulli

$x=0,1$ is a binary variable and $p(x)=f^x(1-f)^{1-x}$. We can write
$$
p(x)=\exp\left(x \log \frac{f}{1-f} +\log (1-f)\right)
$$
Thus $\phi_1(x)=x,\lambda_1=-\log \frac{f}{1-f}, A=-\log(1-f)$.

It is the maximum entropy distribution on a binary variable for a given mean value. Note that in this case the mean determines the distribution completely!

In [ ]:
f = 0.7
num_samples = 5000

samples = bernoulli.rvs(f, size=num_samples)

plt.hist(samples, bins=[-0.5,0.5,1.5], density=True, alpha=0.5, label="frequency")
plt.plot([0,1], [1-f,f], 'o-', label="p")
plt.xticks([0,1])
plt.xlabel('x')
plt.ylabel('p')
plt.title(f'Bernoulli with f={f}')
plt.legend();

print("average:", samples.mean())
print("var", samples.var())

### Examples of exponential family distributions: Gaussian

$$
p(x)\propto \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)\propto \exp\left(\frac{\mu}{\sigma^2}x-\frac{1}{2\sigma^2}x^2\right)=\exp\left(-\lambda_1 \phi_1(x) -\lambda_2\phi_2(x)\right)
$$
with
$$
\phi_1(x)=x,\phi_2(x)=x^2, \lambda_1=-\frac{\mu}{\sigma^2}, \lambda_2 =\frac{1}{2\sigma^2}
$$
It is the maximum entropy distribution on a continuous variable with given mean and variance.

In [ ]:
mean = np.array([1,2])
cov = np.array([[1., 0.4],
                [0.4, 0.5]])
num_samples = 100000

gaussian = multivariate_normal(mean=mean, cov=cov)
detcov = np.linalg.det(cov)

dx = 0.05
lim_std = 2
lims_x = np.array([mean[0] - lim_std, mean[0] + lim_std])
lims_y = np.array([mean[1] - lim_std, mean[1] + lim_std])
xs = np.arange(lims_x[0], lims_x[1], dx)
ys = np.arange(lims_y[0], lims_y[1], dx)

# compute marginals
pxs = np.exp(-0.5*(xs-mean[0])**2/cov[0,0])/np.sqrt(2*np.pi*cov[0,0])
pys = np.exp(-0.5*(ys-mean[1])**2/cov[1,1])/np.sqrt(2*np.pi*cov[1,1])

# set edges for proper histogramming
xedges = np.arange(lims_x[0], lims_x[-1], dx)
yedges = np.arange(lims_y[0], lims_y[-1], dx)
xbins = 0.5 * (xedges[1:] + xedges[:-1])
ybins = 0.5 * (yedges[1:] + yedges[:-1])
xv, yv = np.meshgrid(xbins, ybins)
x_stacked = np.dstack([xv, yv])

# compute density
pbins_true = gaussian.pdf(x_stacked)

# sample from joint distribution
samples = gaussian.rvs(size=num_samples)


# plot stuff
plt.figure(figsize=(12,4))

# plot histogram and joint
plt.subplot(121)
pbins, _, _ = np.histogram2d(samples[:,0], samples[:,1], bins=[xedges, yedges], density=True)
plt.imshow(pbins.T, origin="lower", extent=[xbins[0], xbins[-1],
                                            ybins[0], ybins[-1]], alpha=0.7, cmap="coolwarm", aspect="equal");
plt.contour(xv, yv, pbins_true, levels=3);
plt.xlabel('x')
plt.ylabel('y')

# plot marginals
plt.subplot(122)
labels = ["x", "y"]
for i in range(2):
    plt.hist(samples[:,i], bins=50, density=True, alpha=0.5, color=color_cycle[i], label=labels[i]);
    plt.plot(xs if i==0 else ys, pxs if i==0 else pys, color=color_cycle[i]);
plt.legend();
plt.xlabel('x/y')
plt.ylabel('p(x), p(y)');

## Variational approximation

How do we approximate complex high-dimensional distribution with simpler ones that are easier to handle?

For a given complex distribution $p(x)$, one way to approximate a distribution is to consider a parametrized set of distributions $q(x|\theta)$ and to find 
$$
\theta =\text{argmin}_\theta d(q,p)
$$

with $d(q,p)$ some notion of **distance between distributions**.

## KL divergence

We define the **Relative entropy** or **Kullback-Leibler (KL) divergence** as:

$$
\mathrm{KL}(p||q) = \sum_i p_i \log \left(\frac{ p_i} {q_i} \right)
$$

and for continuous variables:
$$ \mathrm{KL}(p||q) =  - \int p(\mathbf{x})\ln \left( \frac{ q(\mathbf{x})}{p(\mathbf{x})}  \right) d\mathbf{x}
$$

#### Properties:

- $\mathrm{KL}(p||q) \neq \mathrm{KL}(q||p) $ (not a distance)
- $\mathrm{KL}(p||q) \geq 0$, $\mathrm{KL}(p||q) = 0 \Leftrightarrow p = q$ (proof through Jensen's inequality)
- We need that when $q_i=0$ also $p_i=0$, otherwise $p_i \log \left(\frac{ p_i} {q_i} \right)$ is not defined. (The reverse is no problem: when $p_i=0$ and $q_i\ne 0$ we have $p_i \log \left(\frac{ p_i} {q_i} \right)=0$)

### _Intermezzo 1: Convex functions_

Definition of a convex function:

$$\text{$f$ is convex} \iff f(\lambda a + (1-\lambda) b) \leq \lambda f(a) + (1- \lambda)  f(b) \qquad \forall \lambda \in [0,1], \quad \forall a,b$$

- Examples: $f(x) = ax + b$, $f(x) = x^2$, $f(x) = -\ln(x)$ and $f(x) = x \ln (x)$
- Convex: $\cup$ shaped.  Concave: $\cap$ shaped.
- Convex $\Leftarrow$ second derivative non-negative.

In [ ]:
# example of a convex (and not convex!) functions

xs = np.linspace(0.1, 20, 100)

functions = [lambda x : -np.log(x) + 5,
             lambda x : x * np.log(x) + 5,
             lambda x : x**(3/2) + 5,
             lambda x : x**(2/3) + 5]

formulas = ["-log(x) + 5",
            "x log(x) + 5",
            r"$x^{\frac{3}{2}} + 5$",
            r"$x^{\frac{2}{3}} + 5$"]

plt.figure(figsize=(12,4))
cols = ["red", "black"]
for ifunc in range(4):
    f = functions[ifunc]
    formula = formulas[ifunc]
    fs = f(xs)
    
    a, b = 1, 10
    fa, fb = f(a), f(b)
    ts = np.arange(0, 1.05, 0.05)
    chord_x = a * ts + (1 - ts) * b
    chord_y = fa * ts + (1 - ts) * fb

    plt.subplot(1,4,ifunc+1)
    plt.plot(xs, fs, '-', c=color_cycle[0]);
    plt.xlabel('x')
    plt.ylabel('y');
    
    plt.plot(a, fa, 'o', ms=4, c="red")
    plt.plot(b, fb, 'o', ms=4, c="red")

    idx1, idx2 = [0,1] if ifunc < 3 else [1,0]
    
    plt.plot(chord_x, chord_y, c=cols[idx1])
    plt.plot(chord_x, f(chord_x), c=cols[idx2])
    
    plt.vlines(x=a, ymin=fa, ymax=0, ls=':', colors='gray')
    plt.vlines(x=b, ymin=fb, ymax=0, ls=':', colors='gray')
    plt.xticks([a, b], ['a', 'b']);
    plt.title(formula, color=cols[idx2]);

plt.tight_layout();

### _Intermezzo 2: Jensen's inequality_

Convex functions satisfy <em>Jensen's inequality</em> (Proof: see solution of Mackay Ex. 2.14, page 41).
$$
f\left(\sum_{i=1}^M \lambda_i x_i \right) \leq \sum_{i=1}^M \lambda_i f(x_i)
$$
where $\lambda_i \geq 0$, $\sum_i \lambda_i = 1$, for any set points $x_i$.

In other words:
$$f(\langle x \rangle) \leq \langle f(x)\rangle$$


We can use Jensen's inequality to show that $\mathrm{KL}(p||q)\ge 0$ (Proof: see solution of Mackay Ex. 2.26, page 44).

Use $\lambda_i = p_i$, making use of the fact that $-\ln(x)$ is convex:
$$
\mathrm{KL}(p||q) = - \sum_i p_i \ln \left( \frac{q_i}{p_i} \right) \geq
-\ln \left( \sum_i p_i \frac{q_i}{p_i} \right) = -\ln \left(\sum_i q_i\right) = 0
$$

## Back to Variational Approximation

We can thus use KL to define a Variational Approximation:
$$
\theta =\text{argmin}_\theta KL(q||p)\qquad KL(q||p)=\sum_x q(x) \log q(x) -\sum_x q(x) \log p(x)
$$

The variational approximation is extensively used in ML, as we will discuss when we treat the EM algorithm and variational EM algorithms later.

Note that by using $KL(q||p)$ we need to compute expectations w.r.t $q(x)$ and not $p(x)$, which is complex to handle.

In [10]:
# define a function to compute KL divergence
def compute_kl(p0, q0):
    # renormalize distributions
    p = p0 / p0.sum()
    q = q0 / q0.sum()
    plogpq = (p * np.log(p/q))
    KL = plogpq[~np.isnan(plogpq)].sum() # deal with log(1/0)
    return KL

In [ ]:
# An example of how to use KL to measure distance between distributions...
# ... and how to treat normalization with care

# generate probabilities from two Poisson distributions with different rates
mus = [3, 5, 20]

mean_gauss = 20

# compute gaussian densities
ks = np.arange(-10, 50)
gaussian = multivariate_normal(mean=mean_gauss, cov=mean_gauss)
fs_gauss = gaussian.pdf(ks) # gaussian densities
ps_gauss = gaussian.cdf(ks + 0.5) - gaussian.cdf(ks - 0.5) # gaussian probabilities

# get a gaussian with the same mean but a different variance
gaussian_v = multivariate_normal(mean=mean_gauss, cov=7)
fs_gauss_v = gaussian_v.pdf(ks) # gaussian densities
ps_gauss_v = gaussian_v.cdf(ks + 0.5) - gaussian_v.cdf(ks - 0.5) # gaussian probabilities
# compute KL
KL_f_v = compute_kl(fs_gauss, fs_gauss_v) # using densities
KL_p_v = compute_kl(ps_gauss, ps_gauss_v) # using probabilites
KL_f_v_scipy = entropy(fs_gauss, fs_gauss_v) # using densities
KL_p_v_scipy = entropy(ps_gauss, ps_gauss_v) # using probabilites

plt.figure(figsize=(12,4))
plt.subplot(121)

plt.plot(ks, fs_gauss, '.-', ms=3, c="black");
plt.plot(ks, ps_gauss, '.:', ms=3, c="black");

plt.plot(ks, fs_gauss_v, '.:', ms=3, c="gray", label=f"gauss")
plt.plot(ks, ps_gauss_v, '.-', ms=3, c="gray");

KLs_f, KLs_p = np.zeros(len(mus)), np.zeros(len(mus))
KLs_f_scipy, KLs_p_scipy = np.zeros(len(mus)), np.zeros(len(mus))
for imu, mu in enumerate(mus):
    
    # compute probs and plot distribution
    ps_pois = poisson.pmf(ks, mu=mu)
    plt.plot(ks, ps_pois, '.:', c=color_cycle[imu], ms=3, label=f"poiss $\mu$ = {mu}");

    # compute KL divergence with Gaussian (the wrong way)
    KL_f = compute_kl(ps_pois, fs_gauss) # do it yourself
    KL_f_scipy = entropy(ps_pois, fs_gauss) # be lazy and let scipy do it
    KLs_f[imu] = KL_f
    KLs_f_scipy[imu] = KL_f_scipy

    # compute KL divergence with Gaussian (the correct way)
    KL_p = compute_kl(ps_pois, ps_gauss) # do it yourself
    KL_p_scipy = entropy(ps_pois, ps_gauss) # be lazy and let scipy do it
    KLs_p[imu] = KL_p
    KLs_p_scipy[imu] = KL_p_scipy
    
plt.xlabel('k')
plt.ylabel('p(k)')
plt.legend();

plt.subplot(122)
# KL with gaussian
plt.plot(20, KL_f_v, 'x', label="KL gauss approx")
plt.plot(20, KL_p_v, 'x')
# plt.plot(20, KL_f_v_scipy, 'x') # check calculation against scipy
# plt.plot(20, KL_p_v_scipy, 'x') # check calculation against scipy
# KL with poisson
plt.plot(mus, KLs_f, '.-', label="KL Poisson approx")
plt.plot(mus, KLs_p, '.-')
# plt.plot(mus, KLs_f_scipy, '.:') # check calculation against scipy
# plt.plot(mus, KLs_p_scipy, '.:') # check calculation against scipy
plt.xlabel("$\mu$")
plt.ylabel("KL");
plt.legend();

### Example: approximating an elongated Gaussian with an isotropic one

Consider $p$ a elongated Gaussian distribution. Approximate with the simpler factorized distribution $q(z_1,z_2)=q(z_1)q(z_2)$. 

The variational approximation tends to under estimate the variance (Assigment 1.5).

In [ ]:
# define true multivariate gaussian
mean = np.array([0.,0.])
cov = np.array([[1., 0.9],
                [0.9, 1.]])
gaussian = multivariate_normal(mean=mean, cov=cov)
detcov = np.linalg.det(cov)
invcov = np.linalg.inv(cov)

# compute variances of variational approximations
var_kl = 1/invcov[0,0]
var_kl_reverse = cov[0,0]
gaussian_kl = multivariate_normal(mean=mean, cov=np.array([[var_kl, 0.], [0., var_kl]]))
gaussian_kl_reverse = multivariate_normal(mean=mean, cov=np.array([[var_kl_reverse, 0.], [0., var_kl_reverse]]))

# prepare 2d grid of points
dx = 0.05
lim_std = 2
lims_x = np.array([mean[0] - lim_std, mean[0] + lim_std])
lims_y = np.array([mean[1] - lim_std, mean[1] + lim_std])
xs = np.arange(lims_x[0], lims_x[1], dx)
ys = np.arange(lims_y[0], lims_y[1], dx)
xv, yv = np.meshgrid(xs, ys)
x_stacked = np.dstack([xv, yv])

# compute density
pbins = gaussian.pdf(x_stacked)
pbins_kl = gaussian_kl.pdf(x_stacked)
pbins_kl_reverse = gaussian_kl_reverse.pdf(x_stacked)

# plot stuff
plt.figure(figsize=(8,4))

# plot histogram and joint
plt.subplot(121)
plt.imshow(pbins, origin="lower", extent=[xs[0], xs[-1], ys[0], ys[-1]], alpha=0.7, cmap="coolwarm", aspect="equal");
plt.contour(xv, yv, pbins, levels=3);
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(122)
plt.contour(xv, yv, pbins, levels=3);
plt.contour(xv, yv, pbins_kl, levels=3, cmap='Reds');
plt.xlabel('x')
plt.ylabel('y')

plt.tight_layout()

### The reverse $KL(p||q)$ estimates the marginal variance correctly

It is quite easy to understand how the reverse KL estimates the variational approximation. 
\begin{align}
KL(p||q)&=\int dx_1\ldots dx_n p(x_1,\ldots,x_n) \log \frac{p(x_1,\ldots,x_n)}{q_1(x_1)\ldots q_n(x_n)}\\
&=\int dx_1\ldots dx_n p(x_1,\ldots,x_n) \left(\log p(x_1,\ldots,x_n) - \sum_{i=1}^n \log q_i(x_i)\right)\\
&=\text{const}-\sum_{i=1}^n \int dx_1\ldots dx_n p(x_1,\ldots,x_n)\log q_i(x_i)\\
&=\text{const}-\sum_{i=1}^n \int dx_i p_i(x_i)\log q_i(x_i)\\
&=\text{const}+\sum_{i=1}^n KL(p_i||q_i)
\end{align}

So the inverse KL solution is $q_i=p_i$, with $p_i(x_i)=\int dx_{\setminus i} p(x_i,x_{\setminus i})$ the marginal distribution of $p$ on $x_i$.

##### Note that expectations w.r.t $p(x)$ are hard to compute in practice.

In [ ]:
# plot stuff
plt.figure(figsize=(8,4))

# plot histogram and joint
plt.subplot(121)
plt.imshow(pbins, origin="lower", extent=[xs[0], xs[-1], ys[0], ys[-1]], alpha=0.7, cmap="coolwarm", aspect="equal");
plt.contour(xv, yv, pbins, levels=3);
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(122)
plt.contour(xv, yv, pbins, levels=3);
plt.contour(xv, yv, pbins_kl_reverse, levels=3, cmap='Reds')
plt.xlabel('x')
plt.ylabel('y')

plt.tight_layout()

### _Another Intermezzo: Maximum Likelihood_

Given a data set $D=(x_1,\ldots,x_N$) and model $q(x|\theta)$, a common approach to estimate $\theta$ is the so-called **maximum likelihood** method. 

Let us assume each data point is generated independently from $q(x|\theta)$. The probability to observe the entire data set $D$ is
$$
q(D|\theta) = \prod_{i=1}^N q(x_i|\theta)
$$
The best value of $\theta$ is given by maximzing $q(D|\theta)$ w.r.t. $\theta$. Equivalently, one maximizes the log likelihood
$$
L =\frac{1}{N}\log q(D|\theta) = \frac{1}{N}\sum_{i=1}^N \log q(x_i|\theta)
$$

### _KL divergence and Maximum Likelihood_

We can understand the maximum likelihood method in terms of the KL divergence.

Define the <em>empirical distribution</em>
$$
p(x) = \frac{1}{N} \sum_{i=1}^N \delta_{x,x_i} 
$$
Then
$$
KL(p||q) = \sum_x p(x)\log \frac{p(x)}{q(x|\theta)} = -\sum_x p(x)\log q(x|\theta) + \text{const} =-\frac{1}{N}\sum_{i=1}^N \log q(x_i|\theta) 
$$
Thus, minimizing $KL(p||q)$  is equivalent to maximizing the log likelihood.

# <center> Assignments </center>

#### Ex 1.4

Show that the $n$-dimensional multivariate Gaussian distribution
$$
p\left(x|\mu,\Sigma\right)=\frac{1}{\sqrt{\left(2\pi\right)^{n}\det\Sigma}}\exp\left[-\frac{1}{2}\left(x-\mu\right)^{T}\Sigma^{-1}\left(x-\mu\right)\right]
$$
is an exponential family distribution. In particular, the multivariate
Gaussian distribution is the a maximum entropy solution. What are
the constraints?

#### Ex 1.5

Consider a two-dimensional Gaussian distribution with equal variances:
$$
p\left(x_{1},x_{2}\right)\propto\exp\left[-\frac{1}{2}\left(ax_{1}^{2}+ax_{2}^{2}+2bx_{1}x_{2}\right)\right]
$$
where obviously $\left\langle x_{i}\right\rangle _{p}=0$ and $\Sigma=\left\langle xx^{T}\right\rangle _{p}=\left(\begin{array}{cc}
a & b\\
b & a
\end{array}\right)^{-1}$. We would like to approximate it with a product of two-Gaussians:
$$
q\left(x_{1},x_{2}\right)=q_{1}\left(x_{1}\right)q_{2}\left(x_{2}\right)
$$
with
$$
q_{i}\left(x\right)=\frac{e^{-\frac{x^{2}}{2\sigma_{i}^{2}}}}{\sqrt{2\pi\sigma_{i}^{2}}}
$$
We now take the following steps:

* Show that $\int dxq_{i}\left(x\right)\log q_{i}\left(x\right)=-\log\left(\sqrt{2\pi\sigma_{i}^{2}}\right)-\frac{1}{2}$
* Use the previous results to show that
$$
KL\left(q||p\right)=-\sum_{i=1}^{2}\log\left(\sqrt{2\pi\sigma_{i}^{2}}\right)+\frac{a}{2}\sum_{i=1}^{2}\sigma_{i}^{2}+\text{const}
$$
* Show that the variational solution for $q_{i}$ has $\sigma_{i}^{2}=a^{-1}$
* Show that the reverse variational solution solution is given by $\sigma_{i}^{2}=\left\langle x_{i}^{2}\right\rangle _{p}$
* Show that $\left\langle x_{i}^{2}\right\rangle _{p}=\frac{a}{a^{2}-b^{2}}$
and therefore the variance of the reverse solution is large, since
$\frac{1}{a}<\frac{a}{a^{2}-b^{2}}$ (unless $b=0$, in which case
$p$ is already factorized)

## Additional recommended exercises for Lecture 1

Mackay Ex. 2.14, 2.16ab, 2.18, 2.19, 2.26

You can find solutions in the book.